In [1]:
import requests
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

r = requests.get('https://api2.climbzilla.tk/api/v05/top?hall_id=225&sectorId=1867')
flow = r.json()
flow=flow['tops']
flow= str(flow)
pattern = re.compile("'id': \d*")
flow=pattern.findall(flow)
flow=str(flow)
pattern2=re.compile('(\\d+)')
flow=pattern2.findall(flow)
flow=pd.DataFrame(flow)
flow.columns=['id']
flow=flow.id.unique()
flow
ids_1 = pd.DataFrame()
for i  in  flow:
    r = requests.get(f'https://api2.climbzilla.tk/api/v02/top/{i}')
    ids_tmp = r.json()
    try:
        ids_tmp_df = pd.DataFrame({'tops' : ids_tmp['tops'],
                                   'finishes' : [ids_tmp['finishes']]
                                  })
        ids_1 = pd.concat([ids_1, ids_tmp_df])
    except KeyError:
        continue

In [42]:
ids=ids_1
ids=ids.explode('finishes')
ids['finishes']=ids['finishes'].astype(str)
finish =ids['finishes'].str.split(",",expand=True)
ids['top_user']=finish[2]
ids['top_user']=ids['top_user'].map(lambda x: str(x)[11:])

ids['tops']=ids['tops'].astype(str)
new =ids['tops'].str.split(",",expand=True)
ids['id']=new[0]
ids['sector_id']=new[1]
ids['user_id']=new[2]
ids['grade']=new[3]
ids['title']=new[4]
ids['create_time']=new[6]
ids['delete_time']=new[7]
ids.drop(columns=['tops'],inplace=True)
ids['id']=ids['id'].map(lambda x: str(x)[6:])
ids['sector_id']=ids['sector_id'].map(lambda x: str(x)[13:]) #3
ids['user_id']=ids['user_id'].map(lambda x: str(x)[11:]) #4
ids['grade']=ids['grade'].map(lambda x: str(x)[9:]) #2
ids['title']=ids['title'].map(lambda x: str(x)[10:])
ids['create_time']=ids['create_time'].map(lambda x: str(x)[15:-11])
ids['create_time']=pd.to_datetime(ids.create_time)
ids['time_top']=finish[3]
ids['time_top']=ids['time_top'].map(lambda x: str(x)[17:-4])
ids['time_top']=pd.to_datetime(ids.time_top)
ids['delete_time']=ids['delete_time'].map(lambda x: str(x)[17:-12])
ids['delete_time'].fillna(0)

ids.drop(columns =['finishes'], inplace = True)
ids = ids. loc [:, ~ids. columns.str.contains('^Unnamed')]

replacement_grade = {
    " 10": "5A",
    " 11": "5A+",
    " 12": "5B",
    " 13": "5B+",
    " 14": "5C",
    " 15": "5C+",
    " 16": "6A",
    " 17": "6A+",
    " 18": "6B",
    " 19": "6B+",
    " 20": "6C",
    " 21": "6C+",
    " 22": "7A",
    " 23": "7A+",
    " 24": "7B",
    " 25": "7B+",
    " 26": "7C",
    " 27": "7C+",
    " 28": "8A",
    " 29": "8A+",
    " 30": "8B",
    " 31": "8B+",
    " 32": "8C",
    " 33": "8C+",
    " 34": "9A",
}
replacement_setters={
    " 4022":"Дима Беляев",
    " 3176":"Дима Сливинский",
    " 3449":"Дина Ивушкина",
    " 3977":"Паша",
    " 3387":"Рома Зайцев",
    " 12404":"Святослав",
    " 3123":"Таня",
    " 2994":"Ильнур",
    " 4176":"Оля Улыбина",
    " 4796":"Глеб Редькин",
    " 4819":"Егор Стрельников",
    " 5985":"Илья Бурыкин",
    " 11851":"Даниил Бочкин",
    " 10647":"Матвей Неучев",
    " 9510":"Азалия Фаизова",
    " 3398":"Ратмир Фахретдинов",
    " 3391":"Екатерина Заверуха",
    " 9489":"Даниэль Сабитов",
    " 10272":"Маша Заверуха"
}
replacement_users={
    " 4022":"Дима Беляев",
    " 3176":"Дима Сливинский",
    " 3449":"Дина Ивушкина",
    " 3977":"Паша",
    " 3387":"Роман Зайцев",
    " 12404":"Святослав Смирнов",
    " 3123":"Таня Скарлыгина",
    " 2994":"Ильнур Абдрахманов",
    " 4176":"Оля Улыбина",
    " 4796":"Глеб Редькин",
    " 4819":"Егор Стрельников",
    " 5985":"Илья Бурыкин",
    " 11851":"Даниил Бочкин",
    " 10647":"Матвей Неучев",
    " 9510":"Азалия Фаизова",
    " 3398":"Ратмир Фахретдинов",
    " 3391":"Екатерина Заверуха",
    " 9489":"Даниэль Сабитов",
    " 10272":"Маша Заверуха",
    " 2731": "Игорь Поройков",
    " 3004": "Даша Егорова",
    " 12093": "Тамерлан Нагаев",
    " 11838": "Яромир Поройков",
    " 11772": "Даниэль Сабитов",
    " 11612": "Марат Гумеров",
    " 6266": "Арина Шаймухаметова",
    " 6939": "Руслан Оприщенко",
    " 7219": "Степан Борщук",
    " 7568": "Ксения Казеева",
    " 11054": "Данир Шигабутдинов",
    " 11260": "Дима Боровцов",
    " 11493": "Анастасия Торицына",
    " 11579": "lilichu",
    " 12378": "Глеб Кузин",
    " 12459": "Лия Нуриманова",
    " 12464": "Лиана Тимофеева",
    " 12497": "Владислав Оприщенко",
    " 12514": "Алмаз Калимуллин",
    " 12545": "Милана Долгих",
    " 12559": "Валентина Яковлева",
    " 12576": "Диана Нурисламова",
    " 12621": " Игорь",
    " 12710": "\\_(ツ)_/",
    " 12825": "Ролан Давлетбаев",
    " 12872": "Карина Сырватка",
    " 12956": "Кирилл Бикбулатов",
    " 12972": "Неизвестный 1",
    " 12995": "Иван Евдокимов",
    " 12999": "Артем Тараканов",
    " 13003": "Елизавета Чернова",
    " 13004": "Вадим Шарафутдинов",
    " 13012": "Ирина Рыжова",
    " 13014": "Евгений Фетисов",
    " 13015": "Радмила Кадырова",
    " 3132": "Дарья Шаймухаметова",
    " 3151": "Алия Ганиева",
    " 3158": "Рустем Ахмеров",
    " 3376": "Денис Лисицын",
    " 3459": "Даут Зиннуров",
    " 3485": "Павел Лузников",
    " 3634": "Дилара Сайфуллина",
    " 3743": "Адель Романова",
    " 3874": "Руслан Шаймухаметов",
    " 4202": "Максим Никонов",
    " 4696": "Алина Хаятова",
    " 4795": "Татьяна Редькина",
    " 4974": "Елена Сафина",
    " 5720": "Руслан Вильданов",
    " 5828": "Кристина Никонова",
    " 5909": "Антон Семакин",
    " 5963": "Маргарита Селезнева",
    " 5982": "Роберт Галиуллин",
    " 6028": "Елена Рыманова",
    " 6316": "Марат Сафин",
    " 6971": "Альберт Шагиахмедов",
    " 7537": "Никита Бажина",
    " 7562": "Светлана Ахметшина",
    " 8394": "Азамат Нигматуллин",
    " 8451": "Ильяс Хафизов",
    " 8563": "Алексей Ахунов",
    " 8860": "Дмитрий Дорохов",
    " 8883": "Катерина",
    " 9417": "Алена Еремеева",
    " 9587": "Азат март",
    " 1051": "Анна Лагутина",
    " 10593": "Алина Н",
    " 1085": "Алексей Зайкин",
    " 11319": "Арина Файзуллина",
    " 12424": "Игорь Сайфуллин",
    " 12425": "Олег Сайфуллин",
    " 12672": "Дмитрий Шаталин",
    " 13022": "Роман Токунов",
    " 13027": "Яромир Поройков",
    " 13062": "Данте Алигьери",
    " 13064": "Дмитрий Куклин",
    " 13081": "Искандер Валеев",
    " 13089": "Вадим",
    " 13102": "Неизвестный 2",
    " 13107": "Костя Фалин",
    " 13108": "Лиза Архипова",
    " 13111": "Неизвестный 3",
    " 13112": "Светлана",
    " 1874": "Ярослав Тютюнник",
    " 2177": "Ирина Зимарева",
    " 3413": "Руслан Ганеев",
    " 3626": "Яна Габдуллина",
    " 5226": "Вера Сурначева",
    " 5308": "Илья Шлентов",
    " 5910": "Расим Калимуллин",
    " 6101": "Кристаллик",
    " 6323": "Тимур Абдурашитов",
    " 6492": "Айнур Хисамов",
    " 6493": "Вильдан Кужахметов",
    " 6684": "Дмитрий Григорьев",
    " 7425": "Лейла",
    " 7777": "Тимур Гарифуллин",
    " 8020": "Азат Б",
    " 8241": "Михаил Арженцев",
    " 837": "Илья Малышевский",
    " 8960": "Игорь Шибаков",
    " 9050": "Айдар Кагиров",
    " 921": "Алексей Мартьянов",
    " 9285": "Елена Нургалеева"
    

}
ids['grade']=ids['grade'].replace(replacement_grade, regex=True)
ids['user_id']=ids['user_id'].replace(replacement_setters, regex=True)
ids['top_user'] = ids['top_user'].replace(replacement_users, regex = True)
ids=ids.astype({'id': 'int32','sector_id':'int32','user_id':'string','grade':'string','title':'string','top_user':'string'})
ids=ids.query('delete_time==""').drop(columns=['delete_time'])
ids['top_age']=ids['time_top']-ids['create_time']
ids.sort_values('top_user').head(10)


,top_user,id,sector_id,user_id,grade,title,create_time,time_top,top_age
0,,107084,1867,Дима Сливинский,7A+,'Трасса 23',2023-02-12,NaT,NaT
0,Игорь,106852,1867,Дима Сливинский,5C,'Трасса 2 ',2023-02-10,2023-02-10 14:26:03,0 days 14:26:03
0,Игорь,106869,1867,Дима Сливинский,5A,'Трасса 12 ',2023-02-10,2023-02-10 14:11:04,0 days 14:11:04
0,Игорь,106870,1867,Дима Сливинский,6A,'Трасса 13 ',2023-02-10,2023-02-10 14:05:04,0 days 14:05:04
0,Игорь,107394,1867,Дима Сливинский,6A+,'Трасса 34',2023-02-15,2023-02-15 14:25:00,0 days 14:25:00
0,Игорь,106860,1867,Дима Сливинский,5A,'Трасса 5',2023-02-10,2023-02-10 14:12:01,0 days 14:12:01
0,Игорь,106867,1867,Дима Сливинский,5B,'Трасса 10',2023-02-10,2023-02-10 14:14:00,0 days 14:14:00
0,Игорь,106854,1867,Дима Сливинский,6A,'Трасса 3',2023-02-10,2023-02-10 14:29:01,0 days 14:29:01
0,Игорь,106857,1867,Дима Сливинский,6A,'Трасса 4',2023-02-10,2023-02-10 14:33:00,0 days 14:33:00
0,Игорь,106861,1867,Дима Сливинский,6A+,'Трасса 6 ',2023-02-10,2023-02-10 15:20:01,0 days 15:20:01


In [43]:
ids.to_csv('marathon.csv', encoding="utf-8")